## Group Members:
    1 Sarang Padalkar
    2 Priyank Aenugula

## Importing the Libraries.

In [1]:
%autosave 180
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.linear_model import Ridge, RidgeCV, LassoCV
training_data= pd.read_csv('dodgers_training.csv')
test_data= pd.read_csv('dodgers_testing.csv')

Autosaving every 180 seconds


## Data Preprocesssing.

In [2]:
##Encode the categorical variables using the one-hot encoding.
categorical_col=['month', 'day_of_week', 'skies', 'bobblehead']

##Removing all the columns which are not relevant.
for col in training_data.columns:
    if (col not in categorical_col) and (col !='attend' and col != 'temp'):
        training_data.drop(col, axis=1, inplace=True)

for col in test_data.columns:
    if (col not in categorical_col) and (col !='attend' and col != 'temp'):
        test_data.drop(col, axis=1, inplace=True)

In [3]:
##One-Hot Encoding
training_data=pd.get_dummies(training_data,columns=categorical_col,prefix_sep="-")
test_data=pd.get_dummies(test_data,columns=categorical_col,prefix_sep="-")


In [4]:
##Removing the extra columns in test data which are not there in training dataset.
for col in test_data.columns:
    if col not in training_data.columns:
        test_data.drop(col, axis=1, inplace=True)

##Adding the extra columns in test data which are present in training dataset.
for col in training_data.columns:
    if col not in test_data.columns:
        test_data[col] = 0

In [5]:
##Re-organizing the columns for testing data.
cols=training_data.columns.tolist()
test_data= test_data[cols]

print((training_data.info()),(test_data.info()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 20 columns):
attend                   56 non-null int64
temp                     56 non-null int64
month-APR                56 non-null uint8
month-AUG                56 non-null uint8
month-JUL                56 non-null uint8
month-JUN                56 non-null uint8
month-MAY                56 non-null uint8
month-OCT                56 non-null uint8
month-SEP                56 non-null uint8
day_of_week-Friday       56 non-null uint8
day_of_week-Monday       56 non-null uint8
day_of_week-Saturday     56 non-null uint8
day_of_week-Sunday       56 non-null uint8
day_of_week-Thursday     56 non-null uint8
day_of_week-Tuesday      56 non-null uint8
day_of_week-Wednesday    56 non-null uint8
skies-Clear              56 non-null uint8
skies-Cloudy             56 non-null uint8
bobblehead-NO            56 non-null uint8
bobblehead-YES           56 non-null uint8
dtypes: int64(2), uint8(18)
memory us

In [6]:
##Seprating the targets and features from the dataset.
y_train = training_data['attend'].values
X_train = training_data.drop('attend', axis=1).values
X_test = test_data.drop('attend', axis=1).values
y_test = test_data['attend'].values

##Scaling the dataset.
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
print(X_train_scaled)

[[ 0.71678692 -0.43759497  1.91485422 ... -0.49441323  0.31311215
  -0.31311215]
 [-0.41074307 -0.43759497  1.91485422 ... -0.49441323  0.31311215
  -0.31311215]
 [ 0.71678692 -0.43759497  1.91485422 ... -0.49441323 -3.19374388
   3.19374388]
 ...
 [ 2.40808189 -0.43759497 -0.52223297 ... -0.49441323  0.31311215
  -0.31311215]
 [-0.74900206  2.2852182  -0.52223297 ... -0.49441323  0.31311215
  -0.31311215]
 [-0.29799007 -0.43759497 -0.52223297 ... -0.49441323  0.31311215
  -0.31311215]]


<b>1. List the means and standard deviations of the encoded attributes before scaling.</b>

In [8]:
##Observe the dataset has been reduced to unit variance.
print('Orignal mean (Before Scaling) = ', np.mean(X_train,axis=0))
print('Orignal stds (Before Scaling) = ', np.std(X_train,axis=0))

Orignal mean (Before Scaling) =  [7.36428571e+01 1.60714286e-01 2.14285714e-01 8.92857143e-02
 8.92857143e-02 2.32142857e-01 1.78571429e-02 1.96428571e-01
 1.78571429e-01 1.42857143e-01 1.60714286e-01 1.96428571e-01
 8.92857143e-02 8.92857143e-02 1.42857143e-01 8.03571429e-01
 1.96428571e-01 9.10714286e-01 8.92857143e-02]
Orignal stds (Before Scaling) =  [8.86894375 0.36726721 0.4103259  0.2851557  0.2851557  0.42219966
 0.13243212 0.39729635 0.38299305 0.34992711 0.36726721 0.39729635
 0.2851557  0.2851557  0.34992711 0.39729635 0.39729635 0.2851557
 0.2851557 ]


## L2 Regularization.

<b>2. What is the best L2 regularization coefficient? Provide the corresponding linear coefficients.</b>

In [9]:
ridge = RidgeCV(cv=None, store_cv_values=True,alphas=np.linspace(0.1, 1000, 100))
ridge.fit(X_train_scaled, y_train)

print('Best alpha for L2 regularization = ', ridge.alpha_) 
print('Best coefs = ', (ridge.coef_))
print('R2 = ', ridge.score(X_test_scaled,y_test))

Best alpha for L2 regularization =  50.6
Best coefs =  [  515.85380007  -273.56102806   457.54654532  -216.88248689
  1102.57100111  -605.71237164   -75.78927596  -186.41964828
    67.69322043  -835.67182144   490.25315842   238.14165655
  -341.53918562  1070.28517076  -617.19884398   271.19412682
  -271.19412682 -1145.58152754  1145.58152754]
R2 =  0.33678367907497253


## L1 Regularization.

<b>3. What is the best L1 regularization coefficient? Provide the corresponding linear coefficients.</b>

In [23]:
from sklearn.model_selection import cross_val_score
lasso = LassoCV(cv=None, alphas=np.linspace(0.1, 1000, 100))
lasso.fit(X_train_scaled, y_train)
print('Best alpha for L1 regularization = ', lasso.alpha_) 
print('Best coefs = ', (lasso.coef_))
print('R2 = ', lasso.score(X_test_scaled,y_test))

Best alpha for L1 regularization =  616.2
Best coefs =  [ 7.86580396e+02 -0.00000000e+00  6.49814486e+02 -0.00000000e+00
  1.78831458e+03 -6.34820272e+01 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -1.02533744e+03  1.26702529e+02  0.00000000e+00
 -2.71433657e+02  1.32735001e+03 -7.50781549e+02  0.00000000e+00
 -0.00000000e+00 -2.53047199e+03  2.98833973e-12]
R2 =  0.31801946357813315


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2186127.1983425617, tolerance: 194226.85817297298
  tol, rng, random, positive)
D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7171381.275317669, tolerance: 265827.9608432432
  tol, rng, random, positive)


<b>4. What are the predictive attributes selected as a result of L1 regularization?</b>

In [11]:
prominent_attributes={}
feature_names=training_data.drop('attend', axis=1).columns
for x in range(len(lasso.coef_)):
    if abs(lasso.coef_[x])>0:
        prominent_attributes[feature_names[x]]=lasso.coef_[x]
print ("\nProminent_attributes:",prominent_attributes)
df = pd.DataFrame(prominent_attributes,index=range(1))
df.plot.bar(figsize=(16, 5))


Prominent_attributes: {'temp': 786.5803963071144, 'month-AUG': 649.8144860334313, 'month-JUN': 1788.3145848633405, 'month-MAY': -63.48202720103302, 'day_of_week-Monday': -1025.3374368387701, 'day_of_week-Saturday': 126.70252889627709, 'day_of_week-Thursday': -271.43365745305755, 'day_of_week-Tuesday': 1327.3500057241483, 'day_of_week-Wednesday': -750.7815486029613, 'bobblehead-NO': -2530.4719937815066, 'bobblehead-YES': 2.9883397343967633e-12}


The prominant predective attributes are: 
'temp', 'month-AUG', 'month-JUN', 'month-MAY', 'day_of_week-Monday', 'day_of_week-Saturday', 'day_of_week-Thursday', 'day_of_week-Tuesday', 'day_of_week-Wednesday', 'bobblehead-NO', 'bobblehead-YES'

## Linear Moder w/o Regularization.

In [12]:
from sklearn import  linear_model 
reg = linear_model.LinearRegression(fit_intercept=True)
reg.fit(X_train_scaled, y_train)
print('Best coefs = ', (reg.coef_))
print('R2 = ', reg.score(X_test_scaled,y_test))

Best coefs =  [ 1.48616556e+03 -2.83292980e+16 -3.16506469e+16 -2.19955952e+16
 -2.19955952e+16 -3.25665335e+16 -1.02152023e+16 -3.06456070e+16
  3.46699476e+17  3.16766964e+17  3.32463868e+17  3.59647354e+17
  2.58133494e+17  2.58133494e+17  3.16766964e+17 -8.57262940e+15
 -8.57262940e+15 -5.98449438e+16 -5.98449438e+16]
R2 =  0.26189647112959846


<b>5. For the models corresponding to the best L2 and L1 regularization coefficients 
list the following root-mean-square-error (RMSE): (1) RMSE on the training set, (2) 
expected prediction RMSE obtained during the cross-validation, (3) RMSE on the testing 
set. For the linear model without regularization list the following RMSE: (1) RMSE on 
the training set, and (2) RMSE on the testing set.</b>

In [13]:
###   RMSE for L1, L2 and linear regression w/o regularzation models.

from sklearn.metrics import mean_squared_error
y_pred_train_lasso=lasso.predict(X_train_scaled)
y_pred_train_ridge=ridge.predict(X_train_scaled)
y_pred_train_linear=reg.predict(X_train_scaled)

y_pred_test_lasso=lasso.predict(X_test_scaled)
y_pred_test_ridge=ridge.predict(X_test_scaled)
y_pred_test_linear=reg.predict(X_test_scaled)


# L1
print('L1 - RMSE train = ', (np.sqrt(mean_squared_error(y_pred_train_lasso,y_train))))
print('L1 - RMSE test = ', (np.sqrt(mean_squared_error(y_pred_test_lasso,y_test))))

# L2
print('L2 - RMSE train = ', (np.sqrt(mean_squared_error(y_pred_train_ridge,y_train))))
print('L2 - RMSE test = ', (np.sqrt(mean_squared_error(y_pred_test_ridge,y_test))))

print("Expected prediction RMSE for L1 = ",np.min(np.sqrt(np.mean(lasso.mse_path_ ,axis =1))))
print("Expected prediction RMSE for L2 = ",np.min(np.sqrt(np.mean(ridge.cv_values_,axis =0))))

# no regularization
print('noreg - RMSE train = ', (np.sqrt(mean_squared_error(y_pred_train_linear,y_train))))
print('noreg - RMSE test = ', (np.sqrt(mean_squared_error(y_pred_test_linear,y_test))))



L1 - RMSE train =  5568.617925066778
L1 - RMSE test =  6804.850359735517
L2 - RMSE train =  5648.864216385291
L2 - RMSE test =  6710.582066280917
Expected prediction RMSE for L1 =  6753.243824779891
Expected prediction RMSE for L2 =  6737.660948839507
noreg - RMSE train =  5265.924411340254
noreg - RMSE test =  7079.314887836486


<b>6. What observations can you make based on the above RMSE?</b>

The Root mean squared error (RMSE) values are the best way to judge the performance of the models. 
From the above values, we can infer the following:
 
For training data, RMSE value for non-regularaized model is less than both L1(lasso) and L2(ridge) regularized models. However, L1(lasso) model performs better than L2(ridge)model. Thus we can infer that the linear regression model without regularization "over-fits" training data and hence has the lowest RMSE for the trainig dataset.

For testing data, RMSE value for L2(ridge) model is less than both L1(lasso) and non-regularized model. Here the "overfitting" affects the non-regularized model when we use the model for predection on testing dataset. Whereas, we have used regularaization in L2(ridge) model and L1(Lasso) model, that helps them to overcome "overfitting" and perform better than linear non-regularized model. However, L2(ridge) model perfomrs better than L1(lasso model).


If we compare the R2 scores on the testing dataset for the models, we notice that the L2 and L1 model scores are better than the non-regularized models.

<b>7. Predict the attendance on a clear Monday in June when the expected temperature 
is 72 for all three models with and without bobbleheads. Does bobblehead promotion 
have an impact on the attendance?</b>

In [14]:
'''
temp = 72
skies = clear
month = June
day_of_week = monday
bubbleheads = N
'''

X_test_wobubble=scaler.transform(np.array([72,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0]).reshape(1, -1))


y_pred_test_lasso_wob=lasso.predict(X_test_wobubble)
y_pred_test_ridge_wob=ridge.predict(X_test_wobubble)
y_pred_test_linear_wob=reg.predict(X_test_wobubble)

'''
temp = 72
skies = clear
month = June
day_of_week = monday
bubbleheads = Y
'''

X_test_wbubble=scaler.transform(np.array([72,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1]).reshape(1, -1))


y_pred_test_lasso_wb=lasso.predict(X_test_wbubble)
y_pred_test_ridge_wb=ridge.predict(X_test_wbubble)
y_pred_test_linear_wb=reg.predict(X_test_wbubble)




In [15]:
print("Attendence W/o bubblehead:\nL1 - ",y_pred_test_lasso_wob,"\nL2 -",y_pred_test_ridge_wob,"\nNon-Reg -",y_pred_test_linear_wob)
print("\nAttendence with bubblehead:\nL1 - ",y_pred_test_lasso_wb,"\nL2 -",y_pred_test_ridge_wb,"\nNon-Reg -",y_pred_test_linear_wb)

Attendence W/o bubblehead:
L1 -  [41542.63460448] 
L2 - [40638.44252892] 
Non-Reg - [41217.82779728]

Attendence with bubblehead:
L1 -  [50416.63557481] 
L2 - [48673.22150451] 
Non-Reg - [51201.82779728]


<b>Observation</b>: The bubblehead has a positive impact on the Attendance of the game, i.e predicted attendence for a game with bubble head is more than the games without bubblehead.